## Obtaining the data

All steps were conducted in Linux Ubuntu 20.04.3 LTS.

## Create data folder

In [ ]:
!mkdir -p data/brca-data

### Subtype Information

* Copy and paste these commands into RStudio or your R IDE of choice. 
* Move the resulting .tsv file into the same directory as this notebook.

library(tidyverse)

library(TCGAbiolinks)

tcga_subtypes <- PanCancerAtlas_subtypes() %>% filter(cancer.type == "BRCA")

write.table(tcga_subtypes, file='raw_brca_subtypes.tsv', sep='\t', row.names=FALSE)

* The .tsv file with subtype information contains extraneous information and lacks the filename information
required to match it with gene expression data. Therefore we must fix these issues.

* We begin by loading the metadata and extract a dictionary mapping submitter IDs (present in the subtype dataset)
to filenames (present in the gene expression dataset) to bridge feature and label datasets.

In [1]:
import pandas as pd 

brca_subtypes = pd.read_csv('raw_brca_subtypes.tsv', index_col=None, header=0, sep='\t',
                            usecols=['pan.samplesID', 'Subtype_mRNA'])

In [2]:
import json

def get_submitter_filename_dict(metadata):
    with open(metadata) as f:
        metadata_json = json.load(f)
    subm_dict = {entry['associated_entities'][0]['entity_submitter_id']: entry['file_name']
                               for entry in metadata_json
                               if 'TCGA' in entry['associated_entities'][0]['entity_submitter_id']}
    return  subm_dict


In [3]:
submitter_filename_dict = get_submitter_filename_dict('metadata.cart.2021-03-18.json')

brca_subtypes['pan.samplesID'] = brca_subtypes['pan.samplesID'].map(submitter_filename_dict)
brca_subtypes = brca_subtypes.rename(columns={'pan.samplesID': 'filenames'})
brca_subtypes['filenames'] = brca_subtypes['filenames'].dropna()
brca_subtypes = brca_subtypes.set_index('filenames')

print(brca_subtypes.head())

                                                 Subtype_mRNA
filenames                                                    
01f17467-b7de-49d4-a9d7-00108f4de1f9.FPKM.txt.gz       Normal
f5aa7410-7a2e-4267-ba3a-2a6a391f6b45.FPKM.txt.gz         LumA
9075b4ab-16ef-4cdd-bfe5-855d18b15e11.FPKM.txt.gz         LumA
00511204-3512-4a5e-b664-60271e968903.FPKM.txt.gz         LumA
7f92fd47-9938-4069-bc1d-ce6d7d6c1dc4.FPKM.txt.gz         LumA


### Gene expression data

* [GDC Data Portal Repository RNA-Seq Transcriptome Profiling FPKM Files for Primary Site = Breast](https://portal.gdc.cancer.gov/repository?facetTab=cases&filters=%7B%22op%22%3A%22and%22%2C%22content%22%3A%5B%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22cases.primary_site%22%2C%22value%22%3A%5B%22breast%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.analysis.workflow_type%22%2C%22value%22%3A%5B%22HTSeq%20-%20FPKM%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.data_category%22%2C%22value%22%3A%5B%22transcriptome%20profiling%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.data_type%22%2C%22value%22%3A%5B%22Gene%20Expression%20Quantification%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.experimental_strategy%22%2C%22value%22%3A%5B%22RNA-Seq%22%5D%7D%7D%5D%7D)
(last visited 2022/01/10)
* Add All Files to Cart
* Go to the [Cart](https://portal.gdc.cancer.gov/cart) and Remove From Cart > Unauthorized Files
* At the cart, either download directly, or press Download > Manifest. Save the manifest in the same folder as this notebook.
* Still at the cart, press Metadata. Save that JSON file in the same directory as this notebook.
* If you downloaded the manifest, download the [GDC Data Transfer Tool](https://gdc.cancer.gov/access-data/gdc-data-transfer-tool)
(last visited 2022/01/10).

In [ ]:
!wget https://gdc.cancer.gov/files/public/file/gdc-client_v1.6.1_Ubuntu_x64.zip
!unzip gdc-client_v1.6.*zip
!rm gdc-client_v1.6.*.zip wget-log

* If downloading the files using the manifest method, make a directory in which to store them.
* Use the transfer tool to download the files listed in the manifest into that newly create directory.

Note: The download of the transcriptome profiling files will, as a rule, take a while.
Furthermore, if it gives an error, it may be worth re-running the following block of code until all files are downloaded.

In [ ]:
!mkdir -p fpkm-tcga-brca-gene-exp
!./gdc-client download -d fpkm-tcga-brca-gene-exp --manifest gdc_manifest*.txt

* Next, open each individual file and merge them into a single dataframe.
* We will save the concatenated dataframe into a file for future use.

In [4]:
from ..functions.load_gexp_dataset import load_gexp_dataset
load_gexp_dataset(infolder='../data/brca_data/fpkm-tcga-brca-gene-exp', outfile='raw_brca_fpkm.csv')

Because we are only interested in the gene expression levels for data which has BRCA molecular subtype labels
available, we intersect the feature and label datasets, and sort them to ensure that they match on a per-sample basis

In [11]:
raw_brca = pd.read_csv('..data/brca_data/raw_brca_fpkm.csv', index_col=0)

brca_gexp = raw_brca.loc[raw_brca.index.isin(brca_subtypes.index)].sort_index()
brca_subtypes = brca_subtypes.loc[brca_subtypes.index.isin(brca_gexp.index)].sort_index()

print("Gene expression matrix dimensions:", brca_gexp.shape)
print("Subtype vector dimensions:", brca_subtypes.shape)

Gene expression matrix dimensions: (1205, 60483)
Subtype vector dimensions: (1205, 1)


Since coding gene selection is a static preprocessing step, we might perform it during the
one-time data preparation step in order to avoid introducing unnecessary computations when
iterating over and finetuning dynamic steps in the ML pipelines.

To select coding genes:

**Obtaining a list of coding genes**
* [Download a list of protein coding gene IDs from Ensembl](http://www.ensembl.org/biomart/martview/0c0008282d973b80155b23e263f874a8)
(last visited 2022/01/06).
* To select protein coding genes, in Dataset choose Ensemble Genes (Version) > Human Genes (Version);
then click Filters and, under GENE, tick Gene Type and select protein coding;
lastly, go to Attributes, and under GENE untick all boxes except Gene stable ID.
* To download the list of protein coding genes, go to Results,
then Export all results to > File and TSV and tick Unique results only.
Save the file as `protein_coding_genes.txt` in the same directory as this notebook.

**Coding Gene Selection**
* Then load that list and select only coding genes from the main dataframe.

In [8]:
protein_coding_genes = pd.read_csv('protein_coding_genes.txt', sep='\n', header=0).values
unfurled_protein_coding_genes = [gene_id[0] for gene_id in protein_coding_genes.tolist()]
coding_brca_gexp = brca_gexp.loc[:, brca_gexp.columns.str.contains('|'.join(unfurled_protein_coding_genes))]

print("Gene expression matrix dimensions after coding gene selection:", coding_brca_gexp.shape)

Gene expression matrix dimensions after coding gene selection: (1205, 19564)


* Write the processed datasets (both with all genes and with only coding genes) into files.
* CSV files work for pandas.
* For DASK, it is recommended to write files as parquets.
* To facilitate certain downstream pipeline steps, we reset the index names to default ordinal integers.

In [9]:
brca_gexp.to_csv(path_or_buf='../data/brca_data/brca_fpkm.csv', index=False)
coding_brca_gexp.to_csv(path_or_buf='../data/brca_data/coding_brca_fpkm.csv', index=False)

brca_subtypes.to_csv(path_or_buf='../data/brca_data/brca_subtypes.csv', index=False)

* Adjusting chunk size is important. Smaller chunks provide more parallelization, but larger
chunks offer less computational overhead.
* To guarantee that the feature matrix and label vector's samples match, we would like them to have the same
set of sample-wise divisions.

In [10]:
import dask.dataframe as dd
import sys

dask_brca_gexp = dd.from_pandas(brca_gexp.reset_index(drop=True), npartitions=(sys.getsizeof(brca_gexp)//6e7))
dask_coding_brca_gexp = dd.from_pandas(coding_brca_gexp.reset_index(drop=True),
                                       npartitions=(sys.getsizeof(coding_brca_gexp)//6e7))

dask_brca_gexp.to_parquet('../data/brca_data/brca_fpkm.parquet', engine='pyarrow',
                          compression='snappy')
dask_coding_brca_gexp.to_parquet('../data/brca_data/coding_brca_fpkm.parquet',
                                 engine='pyarrow', compression='snappy')

dask_brca_subtypes = dd.from_pandas(brca_subtypes.reset_index(drop=True), npartitions=1).repartition(
    divisions=dask_brca_gexp.divisions)
dask_brca_subtypes.reset_index(drop=True).to_parquet('../data/brca_data/brca_subtypes.parquet',
                                                engine='pyarrow', compression='snappy')

dask_coding_brca_subtypes = dd.from_pandas(brca_subtypes.reset_index(drop=True), npartitions=1).repartition(
    divisions=dask_coding_brca_gexp.divisions)
dask_coding_brca_subtypes.reset_index(drop=True).to_parquet('../data/brca_data/coding_brca_subtypes.parquet',
                                                     engine='pyarrow', compression='snappy')